# tried

1. Adam, lr=.0003, gamma=1e-2
- Adam, lr=0.0001, gamma=1e-2
- Adam, lr=0.0001, beta1=0.5, gamma=1e-2
- Adam, lr=0.0001, beta1=0.5, decoder + noise, mean=0, stddev=1.0
- Adam, lr=0.0001, beta1=0.5, decoder loss = similarity only. 
    - This gets faces early, but quality of faces plateaus as discriminator gets too good. (Might want to try larger training set size than 512).
- Adam, lr=0.0001, beta1=0.5, decoder loss = similarity only.
- Adam, lr=0.0001, beta1=0.5, decoder loss = similarity + 0.1 * disc
    - Both vae and x discriminator out go to 0, decoder wins
- Adam, lr=0.0001, beta1=0.5, decoder loss = similarity + 0.01 * disc
    - Both vae and x discriminator out go to 0, decoder wins
- Adam, lr=0.0001, beta1=0.5, decoder loss = similarity only, training set = 5000
    - Still looks like the discriminator is going to win but the rate at which the discriminator loss drops is slower than in 5 and image output is better.

#### Adam, lr=0.0001, beta1=0.5, training set = 5000
1. decoder loss = similarity only, vae and x noise sigma 10.0
- moved similarity to after 1st conv in discriminator (prev. after 3rd), vae and x noise sigma 1.0, training 2x discr for each generator. 
    - Artifacts of first layer conv and weird coloring (blue and purple). Not sure why. Is there something wrong with the architecture?
- similarity to after 4th conv in discriminator (prev. after 3rd), vae and x noise sigma 1.0, training 2x discr for each generator. 

#### Adam, lr=0.0001, beta1=0.5, training set = 5000, vae and x noise sigma 1.0, training 1x for enc, dec, disc
- decoder loss = similarity + -log(disc(x)), label noise to disc
    - Think I saw mode collapes
- above but change label smoothing so disc 0 labels are 0, disc 1 labels are 0.75


In [1]:
# discriminator loss thresholds:
# below 0.3 stop training disc
# above 0.5 train disc

In [2]:
# Setup

# img_directory = '/Users/rwilliams/Desktop/celeba/training'
img_directory = '/home/ec2-user/training-data/img_align_celeba'
model_save_path = '/home/ec2-user/tf-checkpoints/vaegan-celeba/checkpoint.ckpt'
outputs_directory = '/home/ec2-user/outputs/vaegan-celeba'
log_directory = '/home/ec2-user/tf-logs/vaegan-celeba'

batch_size = 64
training_set_size = 1024
img_size = 64
learning_rate = 0.001
beta1 = 0.9

zsize = 128

# weights similarity loss term for decoder loss
loss_gamma = 0

In [3]:
import numpy as np
import scipy as sp
import os
from utils import imshow, resize_crop, load_img, pixels01, pixels11

In [4]:
# Jupyter imports

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# load training data

# cache results of resizing and cropping on disk
from joblib import Memory
cachedir = '/home/ec2-user/joblib-cache'
memory = Memory(cachedir=cachedir, verbose=0)

@memory.cache
def load_all_imgs(howmany, img_directory):
    training = np.array([resize_crop(load_img(i+1, img_directory), (img_size, img_size)) for i in range(howmany)])
    # rescale each pixel to [-1, 1]. Supposed to help with GANs
    training = pixels11(training)
    return training

training = load_all_imgs(training_set_size, img_directory)

# Build graph

In [6]:
# create models

import tensorflow as tf
from autoencoder import Autoencoder
from discriminator import Discriminator
tf.reset_default_graph()
tf.set_random_seed(42.0)

# input images feed
X = tf.placeholder(tf.float32, [None, img_size, img_size, 3])
R = tf.placeholder(tf.float32, [None, img_size, img_size, 3])


# for feeding random draws of z (latent variable)
Z = tf.placeholder(tf.float32, [None, zsize])

# flags to pass to networks to set batch normalization layers
# as trainable or not
encoder_batch_trainable = tf.placeholder(tf.bool)
decoder_batch_trainable = tf.placeholder(tf.bool)
disc_batch_trainable = tf.placeholder(tf.bool)

# encoder, decoder that will be connected to a discriminator
# vae = Autoencoder(img_shape=(img_size, img_size, 3), zsize=zsize)
# encoder = vae.encoder(X, encoder_batch_trainable)
# decoder = vae.decoder(Z, decoder_batch_trainable)

# a second decoder for decoding samplings of z
# decoder_z_obj = Autoencoder(img_shape=(img_size, img_size, 3), zsize=zsize)
# decoder_z = decoder_z_obj.decoder(Z, decoder_batch_trainable, reuse=True)

# add noise to decoder generated images and inputs
# decoder_plus_noise = decoder + tf.random_normal(
#     shape=(batch_size, img_size, img_size, 3), mean=0.0, stddev=5.0)

# X_plus_noise = X + tf.random_normal(
#     shape=(batch_size, img_size, img_size, 3), mean=0.0, stddev=5.0)

# discriminator attached to vae output
disc_vae_obj = Discriminator(img_shape=(img_size, img_size, 3))
disc_vae_obj.disc(R, disc_batch_trainable)
disc_vae_logits = disc_vae_obj.logits

# discriminator attached to X input
# shares weights with other discriminator
disc_x_obj = Discriminator(img_shape=(img_size, img_size, 3))
disc_x_obj.disc(X, disc_batch_trainable, reuse=True)
disc_x_logits = disc_x_obj.logits

# discriminator attached to random Zs passed through decoder
# shares weights with other discriminator
# disc_z_obj = Discriminator(img_shape=(img_size, img_size, 3))
# disc_z_obj.disc(decoder_z, disc_batch_trainable, reuse=True)
# disc_z_logits = disc_z_obj.logits

# Loss functions and optimizers

In [7]:
# set up loss functions and training_ops

# latent loss used for training encoder
# latent_loss = vae.latent_loss()

# loss that uses decoder to determine similarity between
# actual input images and output images from the vae
# similarity_xentropy = tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=disc_x_obj.similarity, 
#     logits=disc_vae_obj.similarity)
# similarity_loss = tf.reduce_mean(similarity_xentropy)

# create noisy 0, 1 labels for discriminator loss
# clip to [0, 1] because we are treating these as probabilities
# _noisy0 = tf.zeros_like(disc_vae_logits) + tf.random_normal(
#     shape=(batch_size, 1), mean=0.0, stddev=0.2)
# noisy0 = tf.clip_by_value(_noisy0, 0.0, 1.0)

# noisy1 = tf.ones_like(disc_x_logits) - 0.25

# losses for the discriminator's output. Labels are real: 1, fake: 9.
# cross entropy with 0 labels, since training prob that image is fake
disc_vae_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.zeros_like(disc_vae_logits),
    logits=disc_vae_logits))

# cross entropy with 1 labels, since training prob that image is fake
disc_x_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=(tf.ones_like(disc_x_logits) - 0.1), # soft labeling trick
    logits=disc_x_logits))

# cross entropy with 1 labels, since training prob that image is fake
# disc_z_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=tf.ones_like(disc_z_logits),
#     logits=disc_z_logits))

# feed disc output back to decoder. Encoder is trying to spoof disc
# so wants to push outputs of its fake images towards 1.
# decoder_loss_from_disc = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=tf.ones_like(disc_vae_logits),
#     logits=disc_vae_logits))

# minimize these with optimizer
# disc_loss = disc_vae_loss + disc_x_loss + disc_z_loss
disc_loss = disc_vae_loss + disc_x_loss
# encoder_loss = latent_loss + similarity_loss

# decoder_loss = loss_gamma * similarity_loss + decoder_loss_from_disc
# decoder_loss = decoder_loss_from_disc

# get weights to train for each of encoder, decoder, etc.
# pass this to optimizer so it only trains w.r.t the network
# we want to train and just uses other parts of the network as is
# (for example use the discriminator to compute a loss during training
# of the encoder, but don't adjust weights of the discriminator)

# encoder_vars = [i for i in tf.trainable_variables() if 'encoder' in i.name]
decoder_vars = [i for i in tf.trainable_variables() if 'decoder' in i.name]
disc_vars = [i for i in tf.trainable_variables() if 'discriminator' in i.name]

# encoder_update_ops = [i for i in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'encoder' in i.name]
decoder_update_ops = [i for i in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'decoder' in i.name]
disc_update_ops = [i for i in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'discriminator' in i.name]

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
    
# with tf.control_dependencies(encoder_update_ops):
#     train_encoder = optimizer.minimize(encoder_loss, var_list=encoder_vars)
    
# with tf.control_dependencies(decoder_update_ops):
#     train_decoder = optimizer.minimize(decoder_loss, var_list=decoder_vars)

with tf.control_dependencies(disc_update_ops):
    train_disc = optimizer.minimize(disc_loss, var_list=disc_vars)

saver = tf.train.Saver()

# Init session

In [8]:
# create or restore session

sess = tf.InteractiveSession()
try:
    print('trying to restore session')
    saver.restore(sess, model_save_path)
    print('restored session')
except:
    print('failed to restore session, creating a new one')
    tf.global_variables_initializer().run()

# write logs for tensorboard
writer = tf.summary.FileWriter(log_directory, sess.graph)

trying to restore session
INFO:tensorflow:Restoring parameters from /home/ec2-user/tf-checkpoints/vaegan-celeba/checkpoint.ckpt
failed to restore session, creating a new one


In [9]:
# collect data for tensorboard

disc_vae_out = tf.reduce_mean(tf.sigmoid(disc_vae_logits))
disc_x_out = tf.reduce_mean(tf.sigmoid(disc_x_logits))
# disc_z_out = tf.reduce_mean(tf.sigmoid(disc_z_logits))

# tf.summary.scalar('encoder_loss', encoder_loss)
# tf.summary.scalar('decoder_loss', decoder_loss)
tf.summary.scalar('discriminator_loss', disc_loss)
# tf.summary.scalar('similarity_loss', similarity_loss)
# tf.summary.scalar('decoder_loss_from_disc', decoder_loss_from_disc)
# tf.summary.scalar('disc_vae_loss', disc_vae_loss)
# tf.summary.scalar('disc_x_loss', disc_x_loss)
# tf.summary.scalar('disc_z_loss', disc_z_loss)
# tf.summary.scalar('latent_loss', latent_loss)

tf.summary.scalar('disc_vae_out', disc_vae_out)
tf.summary.scalar('disc_x_out', disc_x_out)
# tf.summary.scalar('disc_z_out', disc_z_out)

merged_summary = tf.summary.merge_all()

In [10]:
img_idx = 0

# Train

In [12]:
# write data to tensorboard log
def report():
    xfeed = training[:batch_size]
    zfeed = np.random.normal(size=(batch_size, zsize))
    rfeed = np.random.normal(size=(batch_size, img_size, img_size, 3), loc=0, scale=2.0)
    summary = merged_summary.eval(feed_dict={
        X: xfeed, 
        Z: zfeed,
        R: rfeed,
        encoder_batch_trainable: False,
        decoder_batch_trainable: False,
        disc_batch_trainable: False
    })
    writer.add_summary(summary, epoch)

In [15]:
import math
epochs = 10000
batches = int(float(training_set_size) / batch_size)
print('training over %s batches' % batches)
# number of iterations to train per epoch, so I can easily
# train, e.g. 2x disc for each encoder, decoder; or skip trianing
do_train = {
    'encoder': 0,
    'decoder': 0,
    'disc':    1
}

for epoch in range(epochs):
    print ('epoch %s ' % epoch, end='')
    zdraws = np.random.normal(size=(training_set_size, zsize))
    rdraws = np.random.normal(size=(training_set_size, img_size, img_size, 3), loc=0, scale=2.0)
    
    # train discriminator
    for _ in range(do_train['disc']):
        for batch in range(batches):
            xfeed = training[batch*batch_size:(batch+1)*batch_size]
            zfeed = zdraws[batch*batch_size:(batch+1)*batch_size]
            rfeed = rdraws[batch*batch_size:(batch+1)*batch_size]
            sess.run(train_disc, feed_dict={
                X: xfeed,
                Z: zfeed,
                R: rfeed,
                encoder_batch_trainable: False,
                decoder_batch_trainable: False,
                disc_batch_trainable: True
            })
            print('.', end='')
    report()
    
#     # train encoder
#     for _ in range(do_train['encoder']):
#         for batch in range(batches):
#             xfeed = training[batch*batch_size:(batch+1)*batch_size]
#             sess.run(train_encoder, feed_dict={
#                 X: xfeed,
#                 encoder_batch_trainable: True,
#                 decoder_batch_trainable: False,
#                 disc_batch_trainable: False
#                 })
#             print('.', end='')
        
#     # train decoder
#     for _ in range(do_train['decoder']):
#         for batch in range(batches):
#             xfeed = training[batch*batch_size:(batch+1)*batch_size]
#             zfeed = zdraws[batch*batch_size:(batch+1)*batch_size]
#             sess.run(train_decoder, feed_dict={
#                 X: xfeed, 
#                 Z: zfeed,
#                 encoder_batch_trainable: False,
#                 decoder_batch_trainable: True,
#                 disc_batch_trainable: False
#             })
#             print('.', end='')
#     report()
#     print('')
    
    if (epoch % 1 == 0):
        print('saving session', flush=True)
        saver.save(sess, model_save_path)
                        
#         example = decoder.eval(feed_dict={
#             X: training[:1],
#             Z: zdraws[:1],
#             encoder_batch_trainable: False,
#             decoder_batch_trainable: False,
#             disc_batch_trainable: False
#         })
#         img_save_path = os.path.join(outputs_directory, '%06d.jpg' % img_idx)
#         img_idx += 1
#         sp.misc.imsave(img_save_path, pixels01(example[0]))

training over 16 batches
epoch 0 ................saving session
epoch 1 ................saving session
epoch 2 ................saving session
epoch 3 ................saving session
epoch 4 ................saving session
epoch 5 ................saving session
epoch 6 ................saving session
epoch 7 ................saving session
epoch 8 ................saving session
epoch 9 ................saving session
epoch 10 ................saving session
epoch 11 ................saving session
epoch 12 ................saving session
epoch 13 ................saving session
epoch 14 ................saving session
epoch 15 ................saving session
epoch 16 ................saving session
epoch 17 ................saving session
epoch 18 ................saving session
epoch 19 ................saving session
epoch 20 ................saving session
epoch 21 ................saving session
epoch 22 ................saving session
epoch 23 ................saving session
epoch 24 ................

epoch 202 ................saving session
epoch 203 ................saving session
epoch 204 ................saving session
epoch 205 ................saving session
epoch 206 ................saving session
epoch 207 ................saving session
epoch 208 ................saving session
epoch 209 ................saving session
epoch 210 ................saving session
epoch 211 ................saving session
epoch 212 ................saving session
epoch 213 ................saving session
epoch 214 ................saving session
epoch 215 ................saving session
epoch 216 ................saving session
epoch 217 ................saving session
epoch 218 ................saving session
epoch 219 ................saving session
epoch 220 ................saving session
epoch 221 ................saving session
epoch 222 ................saving session
epoch 223 ................saving session
epoch 224 ................saving session
epoch 225 ................saving session
epoch 226 ......

KeyboardInterrupt: 

In [ ]:
zdraws = np.random.normal(size=(64, zsize))
generated = decoder.eval(feed_dict={
    Z: zdraws,
    encoder_batch_trainable: False,
    decoder_batch_trainable: False,
    disc_batch_trainable: False
})

generated01 = pixels01(generated)

In [ ]:
# imshow(generated01[0:8])
_disc_x = disc_x_obj.disc(X, disc_batch_trainable, reuse=True)
outs = _disc_x.eval(feed_dict={
    X: training[0:64],
    encoder_batch_trainable: False,
    decoder_batch_trainable: False,
    disc_batch_trainable: False
})

In [ ]:
outs[0:10]

In [ ]:
outs[0:10]